In [ ]:
# 41 코루틴 사용하기


calc가 메인 루틴(main routine)이면 add는 calc의 서브 루틴(sub routine)이다. 

In [4]:
def add(a, b):
    c = a+b
    print(c)
    print('add 함수')

def calc():
    add(1, 2)
    print('calc 함수')
    
calc()

3
add 함수
calc 함수


1. 메인 루틴에서 서브루틴을 호출
2. 서브루틴이 실행된 후에 종료되면 서브루틴의 내용은 사라진다.
3. 서브루틴을 실행한 후 메인루틴으로 돌아온다. 
4. 서브루틴은 메인루틴에 종속되어있다. 

###  코루틴 
- 코루틴은 메인루틴, 서브루틴처럼 종속된 관계가 아닌 서로 대등한 관계이다.
- 특정 시점에 상대 코드를 실행한다.  
- 코루틴은 진입점이 여러개이다.
- 진입점(entry point) : 함수의 코드를 실행하는 지점 

## 41.1 코루틴에 값 보내기 
- 코루틴 : 제너레이터의 특별한 형태
- 제너레이터는 yield 값을 발생, 코루틴은 yield 값을 받아온다. 

In [8]:
def number_coroutine():
    while True: # 코루틴을 유지하기 위해 무한 루프 사용 
        x = (yield)
        print(x)
        
co = number_coroutine()
next(co)

co.send(1) # 코루틴에 숫자를 보냄 
co.send(2)
co.send(3)

1
2
3


## 41.2 코루틴 바깥으로 값 전달하기 


In [11]:
def sum_coroutine():
    total = 0 
    while True:
        x = (yield total)  # 코루틴 밖에서 값을 받아오면서 바깥으로 값을 전달 
        total += x
co = sum_coroutine()
print(next(co))

print(co.send(1)) # 코루틴에 숫자를 보냄  : x 에 저장 
print(co.send(2)) # 코루틴에 숫자를 보냄 
print(co.send(3)) # 코루틴에 숫자를 보냄 

0
1
3
6


### 제너레이터와 코루틴의 차이점 
- 제너레이터 : next 함수(__next__ 메서드)를 반복 호출하여 값을 얻어내는 방식 
- 코루틴 : next 함수(__next__ 메서드)를 한 번만 호출한 뒤 send로 값을 주고 받는 방식 

## 41.3 코루틴을 종료하고 예외 처리하기
코루틴은 실행상태를 유지하기 위해 while True: 를 사용하여 동작한다.
강제 종료하고 싶을 땐 close 메서드를 사용한다. 

In [14]:
def number_coroutine():
    while True:
        x = (yield)  # 코루틴 밖에서 값을 받아오면서 바깥으로 값을 전달 
        print(x, end=' ')
co = number_coroutine()
next(co)

for i in range(20):
    co.send(i)
    
co.close()   # 코루틴 종료

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

**GeneratorExit 예외 처리하기**

In [16]:
def number_coroutine():
    try:
        while True:
            x = (yield)  # 코루틴 밖에서 값을 받아오면서 바깥으로 값을 전달 
            print(x, end=' ')
    except:
        print()
        print('코루틴 종료')
        
co = number_coroutine()
next(co)

for i in range(20):
    co.send(i)
    
co.close()   # 코루틴 종료

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
코루틴 종료


## 41.3 코루틴을 종료하고 예외 처리하기


- throw 메서드를 사용함  
- 예외를 코루틴 안으로 throw 한다. 
- throw 메서드에서 지정한 에러 메시지는 except as의 변수에 들어간다. 

In [18]:
def sum_coroutine():
    try:
        total = 0
        while True:
            x = (yield)  # 코루틴 밖에서 값을 받아오면서 바깥으로 값을 전달 
            total += x 
    except RuntimeError as e:
        print(e)
        yield total # 코루틴 밖으로 값을 전달 
        
co = sum_coroutine()
next(co)

for i in range(20):
    co.send(i)
    
print(co.throw(RuntimeError, '예외로 코루틴 끝내기'))

예외로 코루틴 끝내기
190


## 코드 설명

- next(co) : yield 까지의 코루틴안의 코드를 최초 실행한다.  
    코루틴 객체에서 __next__ 함수를 실행해도 된다. 

- range(20)을 반복하면서 0부터 19까지의 숫자(i)를 co.send()로 x 에 전달한다.  
    send() : 코루틴 밖 -> 안 값 전달  
    yield : 코루틴 안 -> 밖 값 전달 

- total에 받아온(send)값을 누적한다.  

- except as e : e라는 변수를 매개변수로 받아 올 수 있다.   
    co.throw(RuntimeError, '예외로 코루틴 끝내기') : '예외로 코루틴 끝내기' 를 e로 받아 print(e)

- yield total : total을 코루틴 밖으로 전달하며, print(co.throw()로 total을 출력함 



## 41.4 하위 코루틴의 반환값 가져오기

In [19]:
def accumulate():
    total = 0
    while True:
        x = (yield)
        if x is None:
            return total   # 코루틴 종료 
            raise StopIteration(total) # StopIteration에 반환할 값을 지정한다. (Python 3.6)
        total += x 

def sum_coroutine():
    while True:
        total = yield from accumulate() # accumulate의 반환값을 가져온다. 
        print(total)

co = sum_coroutine()
next(co)

for i in range(1, 11):
    co.send(i)
co.send(None)

for i in range(1, 101):
    co.send(i)
co.send(None)   

55
5050


## 코루틴 
- 함수가 종료되지 않은 상태에서 값을 주고 받을 수 있는 함수
- 현재의 코드에서 대기, 상대방 코드 실행 을 반복함
- 시간이 오래걸리는 곳에서 사용 : 파일처리, 네트워크처리 등


## 41.6 연습문제: 문자열 검색 코루틴 만들기
다음 소스 코드를 완성하여 문자열에서 특정 단어가 있으면 True, 없으면 False가 출력되게 만드세요. find 함수는 코루틴으로 작성해야 합니다.

In [27]:
def find(word):
    result = False
    while True:
        sentence = (yield result)
        result = word in sentence
        



f = find('Python')
next(f)
 
print(f.send('Hello, Python!'))
print(f.send('Hello, world!'))
print(f.send('Python Script'))
 
f.close()

True
False
True


## 심사문제
표준 입력으로 사칙연산 계산식이 여러 개 입력됩니다. 다음 소스 코드에서 각 계산식의 결과를 구하는 코루틴을 만드세요. 계산식은 문자열 형태이며 값과 연산자는 공백으로 구분됩니다. 그리고 값은 정수로 변환하여 사용하고, 나눗셈은 / 연산자를 사용하세요.

In [ ]:
def calc():
    while True:
        expression = (yield answer)
        
        



expressions = input().split(', ')
 
c = calc()
next(c)
 
for e in expressions:
    print(c.send(e))
 
c.close()